### News Title and Summary

In [166]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [167]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/abhishekdixit/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


In [23]:
# Visit the Mars news site
url = 'https://redplanetscience.com'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [24]:
# Convert the browser html to a soup object and then quit the browserhtml = browser.html
html = browser.html
news_soup = soup(html,'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [25]:
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars"

In [26]:
news_summary = slide_elem.find('div', class_='article_teaser_body').get_text()
news_summary

'In a Q&A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. '

### Featured Images

In [27]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [28]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [29]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [30]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [31]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [32]:
#df = pd.read_html('https://galaxyfacts-mars.com')[0]
df = pd.read_html('https://data-class-mars-facts.s3.amazonaws.com/Mars_Facts/index.html')[0]
df.columns = ['description','Mars','Earth']
df.set_index('description', inplace=True)

In [33]:
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [34]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

### D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

In [168]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [169]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
mars_home_page_url = 'https://marshemispheres.com'

# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
hemi_soup = soup(html,'html.parser')
hemi_box = hemi_soup.find_all('div', class_='description')

for hemi in hemi_box:
    hemi_dict = {}
    link_text = hemi.find('h3')
    img_page = browser.find_by_text(link_text.text)
    img_page.click()
    img_html = browser.html
    img_soup = soup(img_html,'html.parser')
    img_rel_url = img_soup.find('li').find('a').get('href')
    img_full_url = f'{mars_home_page_url}/{img_rel_url}'
    title = img_soup.find('h2', class_='title').get_text()
    hemi_dict = {'img_url': img_full_url, 'title': title}
    hemisphere_image_urls.append(hemi_dict)
    browser.back()



    



In [170]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [171]:
# 5. Quit the browser
browser.quit()